In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.offsetbox as offsetbox
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Polygon
from pathlib import Path
import argparse

In [5]:
def plot_station_xy_Ivan(ax, geometry, station, ratio=0.95):
    station_geometry = geometry[geometry['roll_name'].str.startswith(station)]
    color = 'black'
    for i in range(0, station_geometry.shape[0]):
        chamber = station_geometry.iloc[i]
        center_x = (chamber.x1 + chamber.x2 + chamber.x3 + chamber.x4) / 4
        center_y = (chamber.y1 + chamber.y2 + chamber.y3 + chamber.y4) / 4
        center_z = (chamber.z1 + chamber.z2 + chamber.z3 + chamber.z4) / 4

        p1_p2_x = np.linspace((chamber.x1 * ratio + center_x * (1 - ratio)), (chamber.x2 * ratio + center_x * (1 - ratio)), 10)
        p1_p2_y = np.linspace((chamber.y1 * ratio + center_y * (1 - ratio)), (chamber.y2 * ratio + center_y * (1 - ratio)), 10)
        p2_p3_x = np.linspace((chamber.x2 * ratio + center_x * (1 - ratio)), (chamber.x3 * ratio + center_x * (1 - ratio)), 10)
        p2_p3_y = np.linspace((chamber.y2 * ratio + center_y * (1 - ratio)), (chamber.y3 * ratio + center_y * (1 - ratio)), 10)
        p3_p4_x = np.linspace((chamber.x3 * ratio + center_x * (1 - ratio)), (chamber.x4 * ratio + center_x * (1 - ratio)), 10)
        p3_p4_y = np.linspace((chamber.y3 * ratio + center_y * (1 - ratio)), (chamber.y4 * ratio + center_y * (1 - ratio)), 10)
        p4_p1_x = np.linspace((chamber.x4 * ratio + center_x * (1 - ratio)), (chamber.x1 * ratio + center_x * (1 - ratio)), 10)
        p4_p1_y = np.linspace((chamber.y4 * ratio + center_y * (1 - ratio)), (chamber.y1 * ratio + center_y * (1 - ratio)), 10)

        p_x = np.concatenate((p1_p2_x, p2_p3_x, p3_p4_x, p4_p1_x))
        p_y = np.concatenate((p1_p2_y, p2_p3_y, p3_p4_y, p4_p1_y))

        endcap_chamber_name = chamber.roll_name  
        ax.text(center_x, center_y,
                endcap_chamber_name[5:] + f'\n{center_z:.1f}',
                fontsize=10, c=color, va='center', ha='center', weight='bold')

        ax.plot(p_x, p_y, c=color, lw=3)
    ax.plot([], [], c=color, label='from Ivan', lw=3)
    return ax

def plot_station_xy(ax, geometry, station, ratio=0.95):
    station_geometry = geometry[geometry['roll_name'].str.startswith(station)]
    for i in range(0, station_geometry.shape[0]):
        chamber = station_geometry.iloc[i]
        center_x = (chamber.x1 + chamber.x2 + chamber.x3 + chamber.x4) / 4
        center_y = (chamber.y1 + chamber.y2 + chamber.y3 + chamber.y4) / 4
        center_z = (chamber.z1 + chamber.z2 + chamber.z3 + chamber.z4) / 4

        p1_p2_x = np.linspace((chamber.x1 * ratio + center_x * (1 - ratio)), (chamber.x2 * ratio + center_x * (1 - ratio)), 10)
        p1_p2_y = np.linspace((chamber.y1 * ratio + center_y * (1 - ratio)), (chamber.y2 * ratio + center_y * (1 - ratio)), 10)
        p2_p3_x = np.linspace((chamber.x2 * ratio + center_x * (1 - ratio)), (chamber.x3 * ratio + center_x * (1 - ratio)), 10)
        p2_p3_y = np.linspace((chamber.y2 * ratio + center_y * (1 - ratio)), (chamber.y3 * ratio + center_y * (1 - ratio)), 10)
        p3_p4_x = np.linspace((chamber.x3 * ratio + center_x * (1 - ratio)), (chamber.x4 * ratio + center_x * (1 - ratio)), 10)
        p3_p4_y = np.linspace((chamber.y3 * ratio + center_y * (1 - ratio)), (chamber.y4 * ratio + center_y * (1 - ratio)), 10)
        p4_p1_x = np.linspace((chamber.x4 * ratio + center_x * (1 - ratio)), (chamber.x1 * ratio + center_x * (1 - ratio)), 10)
        p4_p1_y = np.linspace((chamber.y4 * ratio + center_y * (1 - ratio)), (chamber.y1 * ratio + center_y * (1 - ratio)), 10)

        p_x = np.concatenate((p1_p2_x, p2_p3_x, p3_p4_x, p4_p1_x))
        p_y = np.concatenate((p1_p2_y, p2_p3_y, p3_p4_y, p4_p1_y))

        
        endcap_chamber_name = chamber.roll_name.split('_') # [RE+3, R3, CH29, A]
        if endcap_chamber_name[1][1] == "1":
            ax.text(center_x, center_y,
                    f'{endcap_chamber_name[1][-1]}/{endcap_chamber_name[2][2:]}/{endcap_chamber_name[3]}' + f'\n{center_z:.1f}', # 3/29
                    fontsize=5,
                    va='center', ha='center', alpha=0.8)
        else:        
            ax.text(center_x, center_y,
                    f'{endcap_chamber_name[1][-1]}/{endcap_chamber_name[2][2:]}/{endcap_chamber_name[3]}' + f'\n{center_z:.1f}', # 3/29
                    fontsize=6,
                    va='center', ha='center', alpha=0.8)

        if chamber.is_front == True:
            ax.plot(p_x, p_y, c="red", alpha=0.6)
        if chamber.is_front == False:
            ax.plot(p_x, p_y, c="blue", alpha=0.6)
    ax.plot([], [], c='red', label='from CMSSW(front)')
    ax.plot([], [], c='blue', label='from CMSSW(back)')
    return ax

In [6]:
geometry1 = pd.read_csv("/afs/cern.ch/user/j/joshin/public/Workspace-RPC/240716-Geom/csv/rpcf_2026_v4.csv", sep=',', index_col=False)
#geometry2 = pd.read_csv("/afs/cern.ch/user/j/joshin/public/Workspace-RPC/240716-Geom/csv/from_Ivan.csv", sep=',', index_col=False)
geometry2 = pd.read_csv("/afs/cern.ch/user/j/joshin/public/Workspace-RPC/240716-Geom/csv/from_Ivan_240722.csv", sep=',', index_col=False)

output_dir = Path('/afs/cern.ch/user/j/joshin/public/Workspace-RPC/240716-Geom/CompareXY/rpcf_2026_v4')

stations = ['RE-1', 'RE-2', 'RE-3', 'RE-4', 'RE+1', 'RE+2', 'RE+3', 'RE+4']

for station in stations:
    fig, ax = plt.subplots(figsize=(12.8, 10.24))   
    plot_station_xy(ax, geometry1, station, ratio=1.0)
    plot_station_xy_Ivan(ax, geometry2, station, ratio=1.0)
    ax.set_title(f'RPC {station} Geometry', fontsize=18, weight='bold')
    ax.legend(fontsize=14, loc='upper right')
    ax.set_ylabel('Axis-Y [cm]', fontsize=16)
    ax.set_xlabel('Axis-X [cm]', fontsize=16)
    #ax.annotate("(CondDB Info)\nGlobalTag: 140X_mcRun4_realistic_v3\nTag: RPCRECO_Geometry_81YV023",
    #            (0.01, 0.920), xycoords='axes fraction', fontsize=14, horizontalalignment='left') # type: ignore
    ax.annotate("From CMSSW: 2026/v4/rpcf.xml",
                (0.01, 0.950), xycoords='axes fraction', fontsize=14, horizontalalignment='left') # type: ignore
    

    plt.tight_layout()
    if not output_dir.exists(): 
        output_dir.mkdir(parents=True)
    fig.savefig(f'{output_dir}/{station}.png')
    plt.close(fig)